In [1]:
import pandas as pd
import datetime

In [3]:
CLUBS = ['SSG', '키움','LG','KT','KIA', '롯데', '삼성', '두산', '한화', 'NC']
HOLIDAYS = {}
HOLIDAYS['20240410'] = 1
HOLIDAYS['20240505'] = 1
HOLIDAYS['20240518'] = 1
HOLIDAYS['20240606'] = 1
HOLIDAYS['20240815'] = 1

print(len(CLUBS))
print(len(HOLIDAYS))

10
5


## Matches, Calendar, Initial Plan

아래와 같은 형태로 필요

```
  "initialPlan": [
    {
      "index": 0,
      "datetime": "2023-04-01 14:00:00",
      "home": "SSG",
      "away": "KIA",
      "consecutive": 2,
      "weekend": 1,
      "holiday": 2
    }, ...
   ],
  "calendar": [
    {
      "index": 0,
      "datetime": "2023-04-01 14:00:00",
      "consecutive": 2,
      "weekend": 1,
      "holiday": 2
    }, ...
  ],
  "matches": [
    {
      "index": 0,
      "home": "SSG",
      "away": "KIA",
      "consecutive": 2
    }, ...
  ]

```

일단 모두 저장하기

In [31]:
import datetime

initial_plan_arr = []
calendar_arr = []
matches_arr = []

input_sheet = pd.read_excel('./baseball_2024.xlsx', engine='openpyxl', sheet_name="Input")
match_index = 0

matche_object = {}

consecutive =0
day_of_week = 0
holiday_cnt = 0

initial_plan_arr = []
calendar_arr = []
match_arr = []

calendar_index =0


total_match_cnt = 0

for row_index, row in input_sheet.iterrows():
    
    if row['요일'] in ['월'] :
        continue
    
    # date 만들기
    year = 2024
    month = row['월']
    day = row['일']
    

    
    match_date = datetime.date(year, month, day)

    consecutive += 1
    
    
    if row['요일'] in ['토', '일'] :
        day_of_week =1
        holiday_cnt +=1
    elif match_date in HOLIDAYS:
        holiday_cnt +=1  
    
    
    for stadium in ['잠실', '고척', '문학', '수원', '대전', '대구', '광주', '사직', '창원']:
        if not pd.isna(row[stadium])   :
            away_team = str(row[stadium]).split('-')[0]
            home_team = str(row[stadium]).split('-')[1]
            if stadium not in matche_object:
                matche_object[stadium] = {}
                matche_object[stadium]['stadium'] = stadium
                matche_object[stadium]['index'] = match_index
                matche_object[stadium]['datetime'] = match_date
                matche_object[stadium]['home'] = home_team
                matche_object[stadium]['away'] = away_team
        
        
        
   
    
    if row['요일'] in ['목', '일'] : # 항상 목요일 또는 일요일에 끝난다.
        
        calendar_object = {}
        calendar_object['index'] = calendar_index
        calendar_object['datetime'] = match_date
        calendar_object['consecutive'] = consecutive
        calendar_object['weekend'] = day_of_week
        calendar_object['holiday'] = holiday_cnt
        calendar_arr.append(calendar_object)
        calendar_index +=1
        
        for match_object_value in matche_object.values():
            stadium = match_object_value['stadium']
            index = match_object_value['index']
            match_datetime = match_object_value['datetime']
            home = match_object_value['home']
            away = match_object_value['away']
            
            initail_plan_object = {}
            initail_plan_object["index"] = match_index
            initail_plan_object["datetime"] = match_datetime
            initail_plan_object["home"] = home
            initail_plan_object["away"] = away
            initail_plan_object["consecutive"] = consecutive
            initail_plan_object["weekend"] = day_of_week
            initail_plan_object["holiday"] = holiday_cnt
            initial_plan_arr.append(initail_plan_object)
            
          
            
            match_object = {}
            match_object['index'] = match_index
            match_object['home'] = home
            match_object['away'] = away
            match_object['consecutive'] = consecutive
            match_arr.append(match_object)
        
            total_match_cnt +=  consecutive
            match_index +=1
            
            
        matche_object = {}
        consecutive = 0
        day_of_week = 0
        holiday_cnt = 0
        
    
    # if row_index == 9 :
    #     break
        
        
for row in match_arr:
    print(row)
    
print('----')

print(match_index, total_match_cnt)

{'index': 0, 'home': 'LG', 'away': '한화', 'consecutive': 2}
{'index': 1, 'home': 'SSG', 'away': '롯데', 'consecutive': 2}
{'index': 2, 'home': 'KT', 'away': '삼성', 'consecutive': 2}
{'index': 3, 'home': 'KIA', 'away': '키움', 'consecutive': 2}
{'index': 4, 'home': 'NC', 'away': '두산', 'consecutive': 2}
{'index': 5, 'home': 'LG', 'away': '삼성', 'consecutive': 3}
{'index': 6, 'home': 'SSG', 'away': '한화', 'consecutive': 3}
{'index': 7, 'home': 'KT', 'away': '두산', 'consecutive': 3}
{'index': 8, 'home': 'KIA', 'away': '롯데', 'consecutive': 3}
{'index': 9, 'home': 'NC', 'away': '키움', 'consecutive': 3}
{'index': 10, 'home': '두산', 'away': 'KIA', 'consecutive': 3}
{'index': 11, 'home': '키움', 'away': 'LG', 'consecutive': 3}
{'index': 12, 'home': '한화', 'away': 'KT', 'consecutive': 3}
{'index': 13, 'home': '삼성', 'away': 'SSG', 'consecutive': 3}
{'index': 14, 'home': '롯데', 'away': 'NC', 'consecutive': 3}
{'index': 15, 'home': 'LG', 'away': 'NC', 'consecutive': 3}
{'index': 16, 'home': 'SSG', 'away': '두산', '

In [43]:
total_matches = []
index = 0
for club in CLUBS:
    print(club)
    count = 0
    club_matchs = pd.read_excel('./2024_schedule.xlsx', engine='openpyxl', sheet_name=club)
    
    prev_home = ''
    prev_away = ''
    
    consecutive = 0
  
    weekend = 0
    
    for line in club_matchs.iterrows():
        
        row = line[1]
        away = row['경기'].split('vs')[0]
        home = row['경기'].split('vs')[1]
        
        home_yn = True if home == club else False
        if home_yn is False: # 홈경기만 진행
            continue
        
        index +=1
            
        row['날짜']
        month = row['날짜'][:2]
        day = row['날짜'][3:5]
        day_of_week = row['날짜'][6:7]
        hh = row['시간'].hour
        mm = row['시간'].minute
        match_date = datetime.datetime(2023, int(month), int(day), int(hh), int(mm))
        stadium = row['구장']
        
        
        if match_date.strftime('%Y%m%d') in HOLIDAYS :
            print(match_date.strftime('%Y%m%d'))
            holiday_score = HOLIDAYS[match_date.strftime('%Y%m%d')]
        elif day_of_week in ['토', '일'] :
            holiday_score = 1
        else :
            holiday_score = 0

        if prev_home == home and prev_away == away and consecutive <3 :
            consecutive += 1
        else :
            consecutive = 1
            
        prev_home = home
        prev_away = away
        prev_consecutive = consecutive
        # print(day_of_week)
        # print(holiday_score)
        match_dict = {}
        match_dict['index'] = index
        match_dict['home'] = home
        match_dict['away'] = away
        match_dict['consecutive'] = consecutive
        match_dict['match_date'] = match_date
        match_dict['holiday'] = holiday_score
        match_dict['day_of_week'] = day_of_week
        
        total_matches.append(match_dict)

SSG
키움
20230505
20230527
20230606
LG
KT
KIA
20230527
20230606
20230815
롯데
20230505
20230606
20230815
삼성
20230527
20230606
20230815
두산
20230505
20230527
20230606
20230815
한화
20230505
NC
20230505
20230527
20230815


## 모든 데이터를 담기

In [44]:
import csv
myFile = open('total_matches.csv', 'w', encoding='utf8')
writer = csv.writer(myFile)
# writer.writerow(['Name', 'Roll', 'Language'])
for row in total_matches:
    writer.writerow(row.values())
myFile.close()

In [45]:

# ```
#  {
#       "index": 0,
#       "home": "SSG",
#       "away": "두산",
#       "datetime": "2021-04-02 00:00:00",
#       "consecutive": 3,
#       "weekend" : 0,
#       "holiday" : 2,

#     },
# ```
tot_matches_summary = []
index = 0
for k, current_row in enumerate(total_matches):
    # print(index)
    if index >= len(total_matches):
        break
    
    if index == len(total_matches)  -1 or  total_matches[k + 1]['consecutive'] == 1:
        consecutive = current_row['consecutive']
        start_row_index = k - consecutive + 1
        start_row = total_matches[start_row_index]
        
        datetime = start_row['match_date']
        day_of_week = start_row['day_of_week']
        
        home = start_row['home']
        away = start_row['away']
        
        weekend = 0 if start_row['day_of_week'] == '화' else 1 
        holiday = 0
        for i in range(start_row_index, k + 1):
            holiday += total_matches[i]['holiday']
        
        match_dict = {}
        match_dict['index'] = index
        match_dict['home'] = home
        match_dict['away'] = away
        match_dict['consecutive'] = consecutive
        match_dict['datetime'] = datetime
        match_dict['day_of_week'] = day_of_week
        match_dict['weekend'] = weekend
        match_dict['holiday'] = holiday
        tot_matches_summary.append(match_dict)
        
        
    index += 1
            
            
            
        # print(club_matchs.iloc[k + 1])


In [46]:
myFile = open('total_matches_summary.csv', 'w', encoding='utf8')
writer = csv.writer(myFile)
# writer.writerow(['Name', 'Roll', 'Language'])
writer.writerow(['index', 'home', 'away','consecutive', 'datetime', 'day_of_week','weekend', 'holiday'])
for row in tot_matches_summary:
    writer.writerow(row.values())
myFile.close()

## Matches 
  "matches": [
    {
      "index": 0,
      "home": "SSG",
      "away": "두산",
      "consecutive": 3
    },

In [47]:
matchs = []
consecutive_sum = 0
match_no =0 
for i, row in enumerate(tot_matches_summary):
    matchs.append({
        'index' : i,
        'home' : row['home'],
        'away' : row['away'],
        'consecutive': row['consecutive']
    })
    consecutive_sum += row['consecutive']
    match_no +=1
    

In [48]:
print(consecutive_sum) ## 670 (134일 * 5개 경기)

670


In [49]:
myFile = open('matches.csv', 'w', encoding='utf8')
writer = csv.writer(myFile)
# writer.writerow(['Name', 'Roll', 'Language'])
writer.writerow(['index', 'home', 'away','consecutive'])
for row in matchs:
    writer.writerow(row.values())
myFile.close()

## Calendar
"calendar": [
    {
      "index": 0,
      "datetime": "2021-04-02 00:00:00",
      "consecutive": 2,
      "weekend": 1,
      "holiday": 0
    },

In [50]:
calendar_object = []
calendars_duplication_chk = []
for i, row in enumerate(tot_matches_summary):
    if row['datetime'] in calendars_duplication_chk :
        continue
    calendar_object.append({
        'index' : i,
        'datetime': str(row['datetime']),
        'consecutive': row['consecutive'],
        'weekend': row['weekend'],
        'holiday': row['holiday']
    })
    calendars_duplication_chk.append(row['datetime'])


In [51]:
calendar_object[0]

{'index': 0,
 'datetime': '2023-04-01 14:00:00',
 'consecutive': 2,
 'weekend': 1,
 'holiday': 2}

In [52]:
calendar_object = sorted(calendar_object, key=lambda x : x['datetime'] )


In [53]:
myFile = open('calendars.csv', 'w', encoding='utf8')
writer = csv.writer(myFile)
# writer.writerow(['Name', 'Roll', 'Language'])
writer.writerow(['index', 'datetime', 'consecutive','weekend', 'holiday'])
for row in calendar_object:
    writer.writerow(row.values())
myFile.close()


## initial plan

```
  {
      "index": 0,
      "datetime": "2021-04-02 00:00:00",
      "home": "두산",
      "away": "한화",
      "consecutive": 2.0,
      "holidays": 2.0,
    },
```


In [54]:
initial_plans = []
consecutive_sum = 0
for i, row in enumerate(tot_matches_summary):
    initial_plans.append({
        'index' : i,
        'datetime': str(row['datetime']),
        'home': row['home'],
        'away': row['away'],
        'consecutive': row['consecutive'],
        'weekend': row['weekend'],
        'holiday': row['holiday']
    })
    consecutive_sum += row['consecutive']


In [55]:
consecutive_sum

670

In [56]:
myFile = open('initail_plans.csv', 'w', encoding='utf8')
writer = csv.writer(myFile)
# writer.writerow(['Name', 'Roll', 'Language'])
writer.writerow(['index', 'datetime', 'home','away','consecutive','weekend', 'holiday'])
for row in initial_plans:
    writer.writerow(row.values())
myFile.close()

## Basic Info
### teams, distance matrix

In [57]:
import json
with open ('./basic_info.json', 'r', encoding='utf8') as file:
    basic_info = json.load(file)

In [58]:
engine_input = {}
engine_input['initialPlan'] = initial_plans
engine_input['calendar'] = calendar_object
engine_input['matches'] = matchs
engine_input['teams'] = basic_info['teams']
engine_input['distanceMatrix'] = basic_info['distanceMatrix']


In [59]:
engine_input['initialPlan']

[{'index': 0,
  'datetime': '2023-04-01 14:00:00',
  'home': 'SSG',
  'away': 'KIA',
  'consecutive': 2,
  'weekend': 1,
  'holiday': 2},
 {'index': 1,
  'datetime': '2023-04-04 18:30:00',
  'home': 'SSG',
  'away': '롯데',
  'consecutive': 3,
  'weekend': 0,
  'holiday': 0},
 {'index': 2,
  'datetime': '2023-04-14 18:30:00',
  'home': 'SSG',
  'away': 'NC',
  'consecutive': 3,
  'weekend': 1,
  'holiday': 2},
 {'index': 3,
  'datetime': '2023-04-21 18:30:00',
  'home': 'SSG',
  'away': '키움',
  'consecutive': 3,
  'weekend': 1,
  'holiday': 2},
 {'index': 4,
  'datetime': '2023-04-28 18:30:00',
  'home': 'SSG',
  'away': '두산',
  'consecutive': 3,
  'weekend': 1,
  'holiday': 2},
 {'index': 5,
  'datetime': '2023-05-02 18:30:00',
  'home': 'SSG',
  'away': 'KT',
  'consecutive': 3,
  'weekend': 0,
  'holiday': 0},
 {'index': 6,
  'datetime': '2023-05-12 18:30:00',
  'home': 'SSG',
  'away': '한화',
  'consecutive': 3,
  'weekend': 1,
  'holiday': 2},
 {'index': 7,
  'datetime': '2023-05-23 

In [60]:
import json
js = json.dumps(engine_input, indent='\t', ensure_ascii=False)

fp =  open ('json_file2.json', 'w', encoding='utf-8')
fp.write(js)
fp.close()